In [9]:
!pip install python-constraint

  Preparing metadata (setup.py) ... done
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24058 sha256=561ed218ed9d34be1dfcd46f013a1d6b43c5cbb591f778c61c1b7a9c1e0ad3d6
  Stored in directory: /root/.cache/pip/wheels/2e/f2/2b/cb08b5fe129e4f69b7033061f256e5c551b0aa1160c2872aee
Successfully built python-constraint


In [15]:
from constraint import Problem
from itertools import combinations


# Función para generar todos los subconjuntos posibles
def get_subsets(transactions, min_support):
    itemsets = {}
    for transaction in transactions:
        for i in range(1, len(transaction) + 1):
            for subset in combinations(transaction, i):
                subset = frozenset(subset)
                if subset in itemsets:
                    itemsets[subset] += 1
                else:
                    itemsets[subset] = 1
    # Filtrar por soporte mínimo
    frequent_itemsets = {k: v for k, v in itemsets.items() if v >= min_support}
    return frequent_itemsets

# Función para buscar reglas de asociación
def get_association_rules(frequent_itemsets, min_confidence, transactions):
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            for antecedent in combinations(itemset, len(itemset) - 1):
                antecedent = frozenset(antecedent)
                consequent = itemset - antecedent
                antecedent_support = sum(1 for transaction in transactions if antecedent <= transaction)
                confidence = frequent_itemsets[itemset] / antecedent_support
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, confidence))
    return rules

# Función para leer las transacciones
def read_transactions(filename):
    with open(filename, 'r') as file:
        transactions = [line.strip().split(',') for line in file]
    return [frozenset(transaction) for transaction in transactions]

# Ejemplo de uso
if __name__ == "__main__":
    transactions = read_transactions('transactions.txt')
    min_support = 2
    min_confidence = 0.6

    frequent_itemsets = get_subsets(transactions, min_support)
    print("Itemsets frecuentes:", frequent_itemsets)

    association_rules = get_association_rules(frequent_itemsets, min_confidence, transactions)
    print("Reglas de asociación:")
    for rule in association_rules:
        print(f"{set(rule[0])} -> {set(rule[1])} (confianza: {rule[2]:.2f})")

Itemsets frecuentes: {frozenset({'leche'}): 5, frozenset({'mantequilla'}): 4, frozenset({'pan'}): 6, frozenset({'leche', 'mantequilla'}): 3, frozenset({'leche', 'pan'}): 4, frozenset({'mantequilla', 'pan'}): 3, frozenset({'leche', 'mantequilla', 'pan'}): 2}
Reglas de asociación:
{'leche'} -> {'mantequilla'} (confianza: 0.60)
{'mantequilla'} -> {'leche'} (confianza: 0.75)
{'leche'} -> {'pan'} (confianza: 0.80)
{'pan'} -> {'leche'} (confianza: 0.67)
{'mantequilla'} -> {'pan'} (confianza: 0.75)
{'leche', 'mantequilla'} -> {'pan'} (confianza: 0.67)
{'mantequilla', 'pan'} -> {'leche'} (confianza: 0.67)
